In [ ]:
import os

###################################################################################
# Check that these are the correct paths running (change if necessary)

# Finn's Laptop
sourcedataparentfolder = '/mnt/d/KPUM_NODDI/DICOM_KPUM_NODDI'
sourcedatabasefolder = 'sourcedata' # DICOM folder within sourcedataparentfolder
studydir = '/mnt/d/KPUM_NODDI/Data'
niftibasefolder = 'nifti' # nifti basefolder to go within studydir

# KPUM Workstation
sourcedataparentfolder = '/mnt/d/Finn/KPUM_NODDI/DICOM_KPUM_NODDI'
sourcedatabasefolder = 'sourcedata' # DICOM folder within sourcedataparentfolder
studydir = '/mnt/d/Finn/KPUM_NODDI/Data'
niftibasefolder = 'nifti' # nifti basefolder to go within studydir

# tsv-files to keep track of in studydir/niftibasefolder
subjecttrackerpath = os.path.join(studydir, niftibasefolder)
subjecttrackerfile = 'Subject_Tracker_for_sourcedata2nifti.tsv'

###################################################################################
# USER INPUT - START

# Participant details
subject = '' # Subject ID, e.g. '026'
session = '' # Scan (i.e. 'MR1' or 'MR2')

# Define good DCM datafolder as input (and if no data is not good enough or does not exist, leave blank)

## ANATOMY
# FLAIR
flair = '' #e.g. 's03_FLAIR_TRA_FS' 
# T2w Tra
t2wtra = '' #e.g 's04_T2W_TRA'
# High-resolution T1w MPRAGE
t1w_mprage = '' #e.g 's19_MPRAGE'
# T2star
t2star = '' #e.g. 's07_T2starW_COR'
# SWI (if available)
swi = '' #e.g. 's09_SWI_TRA'
# SWI minIP (if available)
swiminip = '' #e.g. 's08_SWI_TRA'
# High-resolution T2w MCRIB (if available)
t2w_mcrib = '' #e.g. 's05_T2W_MCRIB'

## DIFFUSION
# DKI dir-AP data
dki_dirAP = '' #e.g. 's18001_DKI_2.5mm_NODDI_dir-AP_79dir_0_1000_2000'; if multiple folders, use the first one and put 'yes' below
dki_dirAP_multiple_folders = 'yes' #change to 'yes' if the dicomdir2sourcedata-conversion yielded multiple folders, or 'no' if only one folder
# DKI dir-PA data (if available)
dki_dirPA = '' #e.g. 's17001_DKI_2.5mm_NODDI_dir-PA_2dir_0'
dki_dirPA_multiple_folders = 'yes' #change to 'yes' if the dicomdir2sourcedata-conversion yielded multiple folders, or 'no' if only one folder

# USER INPUT - END
###################################################################################

In [ ]:

# Define local functions
def dicom2nifti(inputdicomdir, outputniftidir, outfilebase):
    # Convert DCMs to NIfTIs using dcm2niix
    import subprocess
    from datetime import datetime
    
    print("Start converting DCMs")
    starttime=datetime.now()
    p=subprocess.Popen(f"dcm2niix -a y -d 1 -b y -w 1 -o {outputniftidir} -f {outfilebase} {inputdicomdir}", stdout=subprocess.PIPE, shell=True)
    (output, err) = p.communicate()  
    #This makes the wait possible
    p_status = p.wait()
    #This will give you the output of the command being executed
    #print(f"Command output: {output}")
    endtime=datetime.now()
    print("Done converting DCMs. Elapsed time:", endtime-starttime)


import os
import shutil

def combine_subfolders(input_folder):
    """
    Combines .dcm files from subfolders (e.g., sYY001_"arbitrary_ending", sYY002_"arbitrary_ending")
    into a single folder (e.g., sYY_"arbitrary_ending") and removes the original subfolders.

    Args:
        input_folder (str): The input folder name (e.g., "sYY001_arbitrary_ending").
    """
    # Parse the base folder name (e.g., "sYY" from "sYY001_arbitrary_ending")
    base_folder_name = input_folder.split('_')[0][:-3]  # Extract "sYY" from "sYY001"
    arbitrary_ending = '_'.join(input_folder.split('_')[1:])  # Extract "arbitrary_ending"
    combined_folder_name = f"{base_folder_name}_{arbitrary_ending}"
    combined_folder_path = os.path.join(os.getcwd(), combined_folder_name)

    # Create the combined folder if it doesn't exist
    if not os.path.exists(combined_folder_path):
        os.makedirs(combined_folder_path)
        print(f"Created folder: {combined_folder_path}")

    # Initialize a counter for renaming files
    file_counter = 1

    # Loop through all subfolders matching the pattern
    for subfolder in os.listdir(os.getcwd()):
        if subfolder.startswith(base_folder_name) and subfolder.endswith(arbitrary_ending) and subfolder != combined_folder_name:
            subfolder_path = os.path.join(os.getcwd(), subfolder)

            # Move and rename all .dcm files from the subfolder to the combined folder
            for file in os.listdir(subfolder_path):
                if file.endswith(".dcm"):
                    src_file = os.path.join(subfolder_path, file)
                    dst_file = os.path.join(combined_folder_path, f"{file_counter:05d}.dcm")  # Incremental naming
                    shutil.move(src_file, dst_file)
                    print(f"Moved {src_file} to {dst_file}")
                    file_counter += 1

            # Remove the subfolder completely
            shutil.rmtree(subfolder_path)
            print(f"Removed folder: {subfolder_path}")

    print(f"All .dcm files have been moved to {combined_folder_path}")

# Create conversion file sourcedata2nifti.tsv
import os, subprocess
import pandas as pd

# Define I/O folders and files
sourcedatafolder = os.path.join(sourcedatabasefolder, os.path.join(f'sub-{subject}',f'ses-{session}'))
sourcedatainputfolder = os.path.join(sourcedataparentfolder, sourcedatafolder)
niftifolder = os.path.join(niftibasefolder, os.path.join(f'sub-{subject}',f'ses-{session}'))
niftioutputfolder = os.path.join(studydir, niftifolder)
if not os.path.exists(niftioutputfolder): # then make this directory
    os.makedirs(niftioutputfolder)
subjecttrackertsv = os.path.join(subjecttrackerpath, subjecttrackerfile)

# First, we must check if we have multiple folders for the dki_dir-AP and dki_dir-PA data
if dki_dirAP_multiple_folders == 'yes' :
    # Go to the where the DICOM file folders are
    os.chdir(os.path.join(sourcedatainputfolder))
    combine_subfolders(dki_dirAP)
    base_folder_name = dki_dirAP.split('_')[0][:-3]  # Extract "sYY" from "sYY001"
    arbitrary_ending = '_'.join(dki_dirAP.split('_')[1:])  # Extract "arbitrary_ending"
    dki_dirAP = f"{base_folder_name}_{arbitrary_ending}"
if dki_dirPA_multiple_folders == 'yes' :
    # Go to the where the DICOM file folders are
    os.chdir(os.path.join(sourcedatainputfolder))
    combine_subfolders(dki_dirPA)
    base_folder_name = dki_dirPA.split('_')[0][:-3]  # Extract "sYY" from "sYY001"
    arbitrary_ending = '_'.join(dki_dirPA.split('_')[1:])  # Extract "arbitrary_ending"
    dki_dirPA = f"{base_folder_name}_{arbitrary_ending}"    


# Create list for matching file transfers (DO NOT CHANGE)
sourcedatalist = [t1w_mprage, 
                  t2w_mcrib,
                  t2wtra,
                  flair,
                  t2star,
                  swi,
                  swiminip,
                  dki_dirAP,
                  dki_dirPA ] 
niftidatalist = [os.path.join('anat',f'sub-{subject}_ses-{session}_acq-mprage_T1w.nii'), 
                 os.path.join('anat',f'sub-{subject}_ses-{session}_acq-mcrib_T2w.nii'),
                 os.path.join('anat',f'sub-{subject}_ses-{session}_acq-tra_T2w.nii'),
                 os.path.join('anat',f'sub-{subject}_ses-{session}_FLAIR.nii'),
                 os.path.join('anat',f'sub-{subject}_ses-{session}_acq-t2star_T2starw.nii'),
                 os.path.join('anat',f'sub-{subject}_ses-{session}_acq-swi_T2starw.nii'),
                 os.path.join('anat',f'sub-{subject}_ses-{session}_acq-swi_rec-mnip_T2starw.nii'),
                 os.path.join('dwi',f'sub-{subject}_ses-{session}_dir-AP_dwi.nii'),
                 os.path.join('fmap',f'sub-{subject}_ses-{session}_dir-PA_epi.nii'),
                 ]
# and make this into a DataFrame
sample = {'input': sourcedatalist, 
          'output': niftidatalist,
          'converted': '',
          'comments': ''}
df = pd.DataFrame(sample)

# Locate only the files that are going to be transferred and set this entry to Pending (i.e. not have entry '' in sourcedatalist)
df.loc[df['input'] != '', ['converted']] = 'Pending'
# and save to a LOCAL tsv-file in niftioutputfolder
df.loc[df['converted'] == 'Pending'].to_csv(os.path.join(niftioutputfolder, f'sub-{subject}_ses-{session}_sourcedata2nifti.tsv'), sep="\t", index=False)
# and read df again from the newly written file
df=pd.read_csv(os.path.join(niftioutputfolder, f'sub-{subject}_ses-{session}_sourcedata2nifti.tsv'), sep="\t")
df.fillna('', inplace=True)

# Now loop over these Pending entries in df and perform conversion using dcm2niix
for ind, row in df.loc[df['converted'] == 'Pending'].iterrows() :
    rowdata = row['input']
    inputdicomdir = os.path.join(sourcedatainputfolder, df['input'][ind])
    outfilebase = os.path.splitext(df['output'][ind])[0] # workaround to get rid of file-extension
    print('Converting DCMs in', inputdicomdir ,'and save in', os.path.join(niftioutputfolder, outfilebase))
    # Convert DCMs using local function 
    dicom2nifti(inputdicomdir, niftioutputfolder, outfilebase)
     
    # Update tracker file by writing 'Done'
    df.at[ind, 'converted'] = 'Done'
    df.to_csv(os.path.join(niftioutputfolder, f'sub-{subject}_ses-{session}_sourcedata2nifti.tsv'), sep="\t", index=False)
        
    print('Finished converting DCMs in', inputdicomdir,'and save in', os.path.join(niftioutputfolder, outfilebase))
    print()
        
# Finally, update subjecttrackerfile 
df = pd.read_csv(subjecttrackertsv, sep="\t")
df.fillna('', inplace=True)
# Check if sourcedatafolder is an entry in df, and if not add it to the file
if not df['input'].isin([sourcedatafolder]).any() : 
    # We should add as new entry in the bottom.
    new_row = {'input': sourcedatafolder, 
               'output': niftifolder, 
               'converted': 'Done', 
               'QC': 'Pending', 
               'comments': ''} 
    df.loc[len(df)] = new_row
    # then write to subjecttrackertsv-file
    print(f"{sourcedatafolder} is not in {subjecttrackerfile}. Adding as new entry")
else:
    # update entry/ies 
    print(f"The sub-{subject}_ses-{session} has an entry {sourcedatafolder} is in {subjecttrackerfile}. Updating the rest of the entries")
    df.loc[df['input'] == sourcedatafolder, ['output']] = niftifolder
    df.loc[df['input'] == sourcedatafolder, ['converted']] = 'Done'
    df.loc[df['input'] == sourcedatafolder, ['QC']] = 'Pending'
    df.loc[df['input'] == sourcedatafolder, ['comments']] = ''
# sort the rows according to 'input'
df = df.sort_values(by = 'input')
# and write to subjecttrackertsv-file
df.to_csv(subjecttrackertsv, sep="\t", index=False)